In [36]:
with open('/export/data2/sli/data/MuST-C_synthesized/de/en-de/data/tst_ex_new/txt/tst_ex_new.en') as f:
    tst_lines_new=f.readlines()
with open('/home/sli/DPR/downloads/data/fairseq/train.en') as f:
    train_lines=f.readlines()
with open('/export/data2/sli/data/MuST-C_synthesized/de/en-de/data/terminology/txt/terminology.en') as f:
    term_lines=f.readlines()

In [37]:
import os
import scipy.io.wavfile as wav



def get_audio_path(directory_path):
    # List all files in the directory and filter out those that are .wav files
    wav_files = [f for f in os.listdir(directory_path) if f.endswith('.wav')]
    # Define a function to extract the numeric part from the file name
    def extract_number(file_name):
        # Split the file name by '_' and take the second part, then remove the '.wav' extension and convert to int
        return int(file_name.split('_')[1].replace('.wav', ''))

    # Sort the list of files by the numeric part extracted from each file name
    sorted_list_by_number = sorted(wav_files, key=extract_number)
    audios_path_list = [directory_path+item for item in sorted_list_by_number]

    return audios_path_list

# Path to the directory containing the .wav files
directory_path = '/export/data2/sli/data/MuST-C_synthesized/de/en-de/data/terminology/wav/'
term_audio_files=get_audio_path(directory_path)

tst_audio_files=get_audio_path('/export/data2/sli/data/MuST-C_synthesized/de/en-de/data/tst_ex_new/wav/')


In [38]:
import os
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [39]:
lines_tst_origin=[line.split(' <SEP> ')[1] for line in tst_lines_new]
lines_tst_example=[line.split(' <SEP> ')[0] for line in tst_lines_new]

In [40]:
import torch
import numpy
from dpr_t5.data.biencoder_data_DPR import BiEncoderPassage
from dpr_t5.models import init_biencoder_components
from dpr_t5.options import set_cfg_params_from_state, setup_cfg_gpu, setup_logger

from dpr_t5.utils.data_utils_DPR import Tensorizer
from dpr_t5.utils.model_utils_DPR import (
    setup_for_distributed_mode,
    get_model_obj,
    load_states_from_checkpoint,
    move_to_device,
)

In [41]:
from omegaconf import DictConfig, OmegaConf
cfg = OmegaConf.load("/home/sli/DPR_t5/conf/gen_embs_DPR.yaml")

In [42]:
cfg

{'defaults': [{'encoder': 'hf_bert'}, {'ctx_sources': 'default_sources'}], 'model_file': None, 'ctx_src': None, 'encoder_type': 'ctx', 'out_file': None, 'do_lower_case': True, 'shard_id': 0, 'num_shards': 1, 'batch_size': 32, 'tables_as_passages': False, 'special_tokens': None, 'tables_chunk_sz': 100, 'tables_split_type': 'type1', 'local_rank': -1, 'device': None, 'distributed_world_size': None, 'distributed_port': None, 'no_cuda': False, 'n_gpu': None, 'fp16': False, 'fp16_opt_level': 'O1'}

In [43]:
cfg.encoder= OmegaConf.load("/home/sli/DPR_t5/conf/encoder/hf_bert.yaml")

In [44]:
cfg

{'defaults': [{'encoder': 'hf_bert'}, {'ctx_sources': 'default_sources'}], 'model_file': None, 'ctx_src': None, 'encoder_type': 'ctx', 'out_file': None, 'do_lower_case': True, 'shard_id': 0, 'num_shards': 1, 'batch_size': 32, 'tables_as_passages': False, 'special_tokens': None, 'tables_chunk_sz': 100, 'tables_split_type': 'type1', 'local_rank': -1, 'device': None, 'distributed_world_size': None, 'distributed_port': None, 'no_cuda': False, 'n_gpu': None, 'fp16': False, 'fp16_opt_level': 'O1', 'encoder': {'encoder_model_type': 'hf_bert', 'pretrained_model_cfg': 'bert-base-uncased', 'pretrained_file': None, 'projection_dim': 0, 'sequence_length': 256, 'dropout': 0.1, 'fix_ctx_encoder': False, 'pretrained': True}}

In [45]:
cfg = setup_cfg_gpu(cfg)

2024-02-06 06:15:33,996 [INFO] root: CFG's local_rank=-1
2024-02-06 06:15:33,998 [INFO] root: Env WORLD_SIZE=None
2024-02-06 06:15:34,000 [INFO] root: Initialized host i13hpc59 as d.rank -1 on device=cuda, n_gpu=1, world size=1
2024-02-06 06:15:34,001 [INFO] root: 16-bits training: False 


In [46]:
cfg.model_file='/home/sli/DPR_t5/results_text_text_DPR/dpr_biencoder.35'
saved_state = load_states_from_checkpoint(cfg.model_file)

2024-02-06 06:15:34,012 [INFO] root: Reading saved model from /home/sli/DPR_t5/results_text_text_DPR/dpr_biencoder.35
2024-02-06 06:15:35,849 [INFO] root: model_state_dict keys dict_keys(['model_dict', 'optimizer_dict', 'scheduler_dict', 'offset', 'epoch', 'encoder_params'])


In [47]:
set_cfg_params_from_state(saved_state.encoder_params, cfg)

In [48]:
tensorizer, encoder, _ = init_biencoder_components(cfg.encoder.encoder_model_type, cfg, inference_only=True)
encoder_ctx = encoder.ctx_model
encoder_question = encoder.question_model

2024-02-06 06:15:35,960 [INFO] dpr_t5.models.hf_models: Initializing HF BERT Encoder. cfg_name=bert-base-uncased
2024-02-06 06:15:36,363 [INFO] dpr_t5.models.hf_models: Initializing HF BERT Encoder. cfg_name=bert-base-uncased


In [49]:

encoder_ctx, _ = setup_for_distributed_mode(
        encoder_ctx,
        None,
        cfg.device,
        cfg.n_gpu,
        cfg.local_rank,
        cfg.fp16,
        cfg.fp16_opt_level,
    )
encoder_question, _ = setup_for_distributed_mode(
        encoder_question,
        None,
        cfg.device,
        cfg.n_gpu,
        cfg.local_rank,
        cfg.fp16,
        cfg.fp16_opt_level,
    )
encoder_ctx.eval()
encoder_question.eval()
model_to_load_ctx = get_model_obj(encoder_ctx)
model_to_load_question = get_model_obj(encoder_question)

In [50]:
prefix_len_ctx = len("ctx_model.")
ctx_state = {
        key[prefix_len_ctx:]: value for (key, value) in saved_state.model_dict.items() if key.startswith("ctx_model.")
    }
model_to_load_ctx.load_state_dict(ctx_state, strict=True)

<All keys matched successfully>

In [51]:
prefix_len_question = len("question_model.")
question_state = {
        key[prefix_len_question:]: value for (key, value) in saved_state.model_dict.items() if key.startswith("question_model.")
    }
model_to_load_question.load_state_dict(question_state, strict=False)

<All keys matched successfully>

In [52]:
from dpr_t5.data.biencoder_data_DPR import BiEncoderPassage

In [53]:
all_passages=[('term_set',BiEncoderPassage(term_lines[i],'')) for i in range(len(term_lines))]

In [54]:
results=[]
model=encoder_ctx
insert_title= True
for passage in all_passages:
    batch=[passage]
    batch_token_tensors = [tensorizer.text_to_tensor(ctx[1].text, title=ctx[1].title if insert_title else None) for ctx in batch]
    ctx_ids_batch = move_to_device(torch.stack(batch_token_tensors, dim=0), cfg.device)
    ctx_seg_batch = move_to_device(torch.zeros_like(ctx_ids_batch), cfg.device)
    ctx_attn_mask = move_to_device(tensorizer.get_attn_mask(ctx_ids_batch), cfg.device)
    with torch.no_grad():
            _, out, _ = model(ctx_ids_batch, ctx_seg_batch, ctx_attn_mask)
    out=out.cpu()
    results.extend([out[0].view(-1).numpy()])

In [55]:
print(len(results))
print(results[0].shape)

9821
(768,)


In [56]:
all_questions=[('tst_set',BiEncoderPassage(lines_tst_origin[i],'')) for i in range(len(lines_tst_origin))]

In [57]:
results_question=[]
model=encoder_question
for question in all_questions:
    batch=[question] 
    batch_token_tensors = [tensorizer.text_to_tensor(ctx[1].text, title=ctx[1].title if insert_title else None) for ctx in batch]
    ctx_ids_batch = move_to_device(torch.stack(batch_token_tensors, dim=0), cfg.device)
    ctx_seg_batch = move_to_device(torch.zeros_like(ctx_ids_batch), cfg.device)
    ctx_attn_mask = move_to_device(tensorizer.get_attn_mask(ctx_ids_batch), cfg.device)
    with torch.no_grad():
            _, out, _ = model(ctx_ids_batch, ctx_seg_batch, ctx_attn_mask)
    out=out.cpu()
    results_question.extend([out[0].view(-1).numpy()])

In [58]:
print(len(results_question))
print(results_question[0].shape)

2500
(768,)


In [59]:
tst_tensors=[torch.unsqueeze(torch.from_numpy(results_question[i]),dim=0) for i in range(len(results_question))]
terminology_tensors=[torch.unsqueeze(torch.from_numpy(results[i]),dim=0) for i in range(len(results))]

In [60]:
print(tst_tensors[0].shape)
print(terminology_tensors[0].shape)

torch.Size([1, 768])
torch.Size([1, 768])


In [61]:
terminology_data=torch.cat(terminology_tensors,dim=0)
tst_term_pairs=[]
#for j,item in enumerate(tst_tensors):
#    dist=torch.norm(terminology_data-item,dim=1,p=2)
#    knn=dist.topk(5,largest=False)
#    #knn.values
#    tst_term_pairs.append([j,knn.indices.tolist(),knn.values])
for j,item in enumerate(tst_tensors):
    dist=torch.matmul(terminology_data,torch.transpose(item,0,1))
    dist=torch.transpose(dist,0,1)   
    knn=dist.topk(10)
    #knn.values
    tst_term_pairs.append([j,knn.indices.tolist()[0],knn.values])
with open('/home/sli/DPR_t5/retrieve_results/tst_term_pairs_dpr_q_n_p_text_text_10.txt','w') as f:
        for item in tst_term_pairs:
               f.write('{} \n'.format(item))


In [62]:
for i,pair in enumerate(tst_term_pairs):
    if i==0:
        with open('/home/sli/DPR_t5/retrieve_results/tst_with_retrieved_example_dpr_q_n_p_text_text_1st_NN_3.txt','w') as g: #
            g.write(term_lines[pair[1][0]].strip()+' <SEP> '+lines_tst_origin[pair[0]])
        with open('/home/sli/DPR_t5/retrieve_results/tst_with_retrieved_example_dpr_q_n_p_text_text_2st_NN_3.txt','w') as g: #
            g.write(term_lines[pair[1][1]].strip()+' <SEP> '+lines_tst_origin[pair[0]])
        with open('/home/sli/DPR_t5/retrieve_results/tst_with_retrieved_example_dpr_q_n_p_text_text_3st_NN_3.txt','w') as g: #
            g.write(term_lines[pair[1][2]].strip()+' <SEP> '+lines_tst_origin[pair[0]])
    else:
        with open('/home/sli/DPR_t5/retrieve_results/tst_with_retrieved_example_dpr_q_n_p_text_text_1st_NN_3.txt','a') as g: #
            g.write(term_lines[pair[1][0]].strip()+' <SEP> '+lines_tst_origin[pair[0]])
        with open('/home/sli/DPR_t5/retrieve_results/tst_with_retrieved_example_dpr_q_n_p_text_text_2st_NN_3.txt','a') as g: #
            g.write(term_lines[pair[1][1]].strip()+' <SEP> '+lines_tst_origin[pair[0]])
        with open('/home/sli/DPR_t5/retrieve_results/tst_with_retrieved_example_dpr_q_n_p_text_text_3st_NN_3.txt','a') as g: #
            g.write(term_lines[pair[1][2]].strip()+' <SEP> '+lines_tst_origin[pair[0]])


In [63]:

with open('/home/sli/DPR_t5/retrieve_results/tst_with_retrieved_example_dpr_q_n_p_text_text_1st_NN_3.txt',encoding='utf-8') as f:
    lines = f.readlines()
import json
with open('/home/sli/DPR/downloads/data/fairseq/rareword_terminology.txt') as f:
    data = f.read()
dictionary=json.loads(data)
import spacy
  
nlp=spacy.load("en_core_web_sm") 
examples=[]
sentences=[]
for line in lines:
    id_sep=line.find(' <SEP> ')
    example=line[:id_sep]
    sentence=line[id_sep+7:]
    examples.append(example)
    sentences.append(sentence)
sentence_rarewords={}
example_rarewords={}

for i in range(len(examples)):
    line_sentence=sentences[i]
    line_example=examples[i]
    
    sentence_rarewords[i]=[]
    example_rarewords[i]=[]
    
    doc_sentence=nlp(line_sentence)
    pred_item_sentence=[token.lemma_.lower() for token in doc_sentence if not token.is_stop and not token.is_punct]
    
    doc_example=nlp(line_example)
    pred_item_example=[token.lemma_.lower() for token in doc_example if not token.is_stop and not token.is_punct]
    pred_item_example_new=" ".join(pred_item_example)
    for j,item in enumerate(pred_item_sentence):
        try:
            dictionary[item]
            sentence_rarewords[i].append(item)

            if item in pred_item_example_new:
                example_rarewords[i].append(item)

        except:
            continue
    
total_rarewords=[]
for k,v in sentence_rarewords.items():
    total_rarewords=total_rarewords+v
    total_rarewords_set=set(total_rarewords)
    total_rarewords=list(total_rarewords)
    
    total_example_rarewords=[]
    for k,v in example_rarewords.items():
        total_example_rarewords=total_example_rarewords+v
    total_example_rarewords_set=set(total_example_rarewords)
    total_example_rarewords=list(total_example_rarewords)
    
print("num of rare words is {}".format(len(total_rarewords))) #num of unrecognized lemma
print("num of rare words in example is {}".format(len(total_example_rarewords))) #num of recognized word


num of rare words is 2959
num of rare words in example is 1032


In [64]:

with open('/home/sli/DPR_t5/retrieve_results/tst_with_retrieved_example_dpr_q_n_p_text_text_2st_NN_3.txt',encoding='utf-8') as f:
    lines = f.readlines()
import json
with open('/home/sli/DPR/downloads/data/fairseq/rareword_terminology.txt') as f:
    data = f.read()
dictionary=json.loads(data)
import spacy
  
nlp=spacy.load("en_core_web_sm") 
examples=[]
sentences=[]
for line in lines:
    id_sep=line.find(' <SEP> ')
    example=line[:id_sep]
    sentence=line[id_sep+7:]
    examples.append(example)
    sentences.append(sentence)
sentence_rarewords={}
example_rarewords={}

for i in range(len(examples)):
    line_sentence=sentences[i]
    line_example=examples[i]
    
    sentence_rarewords[i]=[]
    example_rarewords[i]=[]
    
    doc_sentence=nlp(line_sentence)
    pred_item_sentence=[token.lemma_.lower() for token in doc_sentence if not token.is_stop and not token.is_punct]
    
    doc_example=nlp(line_example)
    pred_item_example=[token.lemma_.lower() for token in doc_example if not token.is_stop and not token.is_punct]
    pred_item_example_new=" ".join(pred_item_example)
    for j,item in enumerate(pred_item_sentence):
        try:
            dictionary[item]
            sentence_rarewords[i].append(item)

            if item in pred_item_example_new:
                example_rarewords[i].append(item)

        except:
            continue
    
total_rarewords=[]
for k,v in sentence_rarewords.items():
    total_rarewords=total_rarewords+v
    total_rarewords_set=set(total_rarewords)
    total_rarewords=list(total_rarewords)
    
    total_example_rarewords=[]
    for k,v in example_rarewords.items():
        total_example_rarewords=total_example_rarewords+v
    total_example_rarewords_set=set(total_example_rarewords)
    total_example_rarewords=list(total_example_rarewords)
    
print("num of rare words is {}".format(len(total_rarewords))) #num of unrecognized lemma
print("num of rare words in example is {}".format(len(total_example_rarewords))) #num of recognized word



num of rare words is 2959
num of rare words in example is 223
